In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
import time

import pandas as pd
import re
import os

In [31]:
# 크롬드라이버 위치 절대경로로 설정
driver = webdriver.Chrome('/Users/yangsemo/chromedriver')
driver.implicitly_wait(10) # seconds
page = 1
url = f'https://myfranchise.kr/category/contents/%EC%B9%B4%ED%8E%98%C2%B7%EB%94%94%EC%A0%80%ED%8A%B8?page={page}'
driver.get(url)



/var/folders/xy/wq52q6wn4ts_3hz5xm2fj5_80000gn/T/ipykernel_32364/900032609.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/yangsemo/chromedriver')


In [23]:
x = 1 # page down을 위함
page = 1

# dataframe 생성
col = ['브랜드명','매장수','상호', '대표자', '법인설립등기일', '사업자등록일', '대표번호', '대표팩스번호', '사업자유형', '법인등록번호', '사업자등록번호', '주소']
df_a = pd.DataFrame(columns=col)

# 한페이지에 나와있는 카페 수
cafe_len = len(driver.find_elements(By.CLASS_NAME, "css-h9u7nh"))
print('cafe 개수: ', cafe_len)

while(True):
    try:
        print('if문 확인')
        if x == cafe_len+1:
            x=1
            page+=1
            
            print('------------page 변경--------------')
            print('page: ', page)
            print('----------------------------------')
            
            url = f'https://myfranchise.kr/category/contents/%EC%B9%B4%ED%8E%98%C2%B7%EB%94%94%EC%A0%80%ED%8A%B8?page={page}'
            driver.get(url)
            time.sleep(2)
            
            # 한페이지에 나와있는 카페 수
            # 마지막 페이지 카페 수는 7개 이므로 cafe_len 갱신 필요
            cafe_len = len(driver.find_elements(By.CLASS_NAME, "css-h9u7nh"))
#             print('Next Page cafe 개수: ', cafe_len)

            if page == 57:
                break

        else:
            for i in range(cafe_len):
                
                # 본사 정보 Data List
                val = []
                
                # 브랜드명
                cafe = driver.find_elements(By.CLASS_NAME, "css-1w2bh6w")[i]

                cafe_name = cafe.find_element(By.CLASS_NAME, "css-1xkzyxt")
                cafe_count = cafe.find_element(By.CLASS_NAME, "css-1wz331c")

                cafe_name_text = cafe_name.get_attribute("innerText")
                
                cafe_count_text1 = cafe_count.get_attribute("innerText").split(' ')[1]
                cafe_count_text2 = re.sub(r'[^0-9]', '', cafe_count_text1)
                
                
                val.append(cafe_name_text)
                val.append(cafe_count_text2)
                print('번호:', x, ' 브랜드명:',cafe_name_text, ' 매장수:',cafe_count_text2)
                cafe.click()
                
                time.sleep(2.5)
                

                # 본사 탭 클릭
                
                bonsa = driver.find_element('xpath', '//*[@id="brand-relative-tabs-mobile"]/div[1]')
                length = len(bonsa.find_elements(By.TAG_NAME, 'button'))
                bonsa_btn = bonsa.find_elements(By.TAG_NAME, 'button')[-1]
                bonsa_btn.click()
                
                time.sleep(1)

                
                # 본사 정보                
                company = driver.find_element(By.CLASS_NAME, 'css-p43zmh')


                for val_name in company.find_elements(By.CLASS_NAME, "css-26ughq"):
                    val.append(val_name.get_attribute("innerText"))
                    
                # 주소만 class_name이 다르므로 따로 추가
                address = company.find_element(By.CLASS_NAME, "css-m8flp7")
                address_name = address.get_attribute('innerText')
                val.append(address_name)
            
                # 카페 인덱스 증가
                x+=1
                
                # 데이터 삽입
                df_a.loc[len(df_a)] = val

                driver.back()
                time.sleep(2)

    # 예외처리로 데이터 못 불러오면 pagedown
    except Exception as error:
        print('error:', error)
        body = driver.find_element(By.CSS_SELECTOR,'body')
        body.send_keys(Keys.PAGE_DOWN)

        
df_a.to_csv('cafe_info.csv', index=False)

cafe 개수:  24
if문 확인
번호: 1  브랜드명: 요거트바이피플  매장수: 
번호: 2  브랜드명: 브래드바이피플  매장수: 
번호: 3  브랜드명: 피자바이피플  매장수: 
번호: 4  브랜드명: 지브릭커피(G,BRICK COFFEE)  매장수: 6
번호: 5  브랜드명: 배달의카페  매장수: 5
번호: 6  브랜드명: 브리오슈도레(BRIOCHE DOREE)  매장수: 21
번호: 7  브랜드명: WSTYLE BAGLE  매장수: 
번호: 8  브랜드명: WSTYLE BAKERY CAFE  매장수: 
번호: 9  브랜드명: 레드인  매장수: 
번호: 10  브랜드명: 커피101스트릿(COFFEE 101 STREET)  매장수: 15
번호: 11  브랜드명: 레이어드  매장수: 
번호: 12  브랜드명: 젤라뚜또  매장수: 11
번호: 13  브랜드명: 노커커피  매장수: 
번호: 14  브랜드명: 오슬로우(OHSLOW)  매장수: 14
번호: 15  브랜드명: 커피바이브  매장수: 
번호: 16  브랜드명: 한스(HANS)  매장수: 35
번호: 17  브랜드명: 닥터세이지(Dr.SAGY)  매장수: 
번호: 18  브랜드명: 글린공원  매장수: 
번호: 19  브랜드명: 와플덴  매장수: 
번호: 20  브랜드명: 아운(āoon)  매장수: 1
번호: 21  브랜드명: 후즈티(WHO'S TEA)  매장수: 5
번호: 22  브랜드명: 박사단팥빵  매장수: 7
번호: 23  브랜드명: 식빵연구소  매장수: 
번호: 24  브랜드명: 꽈배기브라더스  매장수: 4
if문 확인
------------page 변경--------------
page:  21
----------------------------------
if문 확인
번호: 1  브랜드명: 프랭크서울  매장수: 27
번호: 2  브랜드명: 홉슈크림  매장수: 11
번호: 3  브랜드명: 쿠소이  매장수: 1
번호: 4  브랜드명: D’ THREE(디쓰리)  매장수: 
번호: 5  브랜드명: 도

if문 확인
번호: 1  브랜드명: 커피는  매장수: 
번호: 2  브랜드명: 이너리트  매장수: 1
번호: 3  브랜드명: TRDLO  매장수: 
번호: 4  브랜드명: 레드애플  매장수: 12
번호: 5  브랜드명: 감동카페  매장수: 
번호: 6  브랜드명: 지구별관  매장수: 
번호: 7  브랜드명: 블루포트(BLUEPOT)  매장수: 
번호: 8  브랜드명: 카페 인기가요  매장수: 
번호: 9  브랜드명: 미드테이블커피 (MIDT COFFEE)  매장수: 
번호: 10  브랜드명: 라고바움  매장수: 
번호: 11  브랜드명: 크로플덕  매장수: 
번호: 12  브랜드명: 또봉이핫독&도넛  매장수: 
번호: 13  브랜드명: 1LiTER COFFEE(일리터커피)  매장수: 6
번호: 14  브랜드명: 고로케하우스  매장수: 3
번호: 15  브랜드명: 프레쉬앤리프레쉬(fresh&refresh)  매장수: 
번호: 16  브랜드명: KUNGFU TEA(쿵푸티)  매장수: 7
번호: 17  브랜드명: 머지커피  매장수: 
번호: 18  브랜드명: 도노텀블  매장수: 10
번호: 19  브랜드명: 더 달콤  매장수: 
번호: 20  브랜드명: 밀크로플  매장수: 1
번호: 21  브랜드명: 히즈빈스(HISBEANS)  매장수: 16
번호: 22  브랜드명: 우리동네라운지  매장수: 
번호: 23  브랜드명: 오가다  매장수: 62
번호: 24  브랜드명: 백억커피  매장수: 
if문 확인
------------page 변경--------------
page:  31
----------------------------------
if문 확인
번호: 1  브랜드명: 차뜨라므  매장수: 3
번호: 2  브랜드명: 퐁커피  매장수: 
번호: 3  브랜드명: 월화수카페  매장수: 
번호: 4  브랜드명: 카페마엔코  매장수: 3
번호: 5  브랜드명: 커피볶는집  매장수: 3
번호: 6  브랜드명: 드립드랍(DRIPDROP)  매장수: 
번호: 7  브랜드명: 부

번호: 4  브랜드명: 하와더카페  매장수: 
번호: 5  브랜드명: 마노핀  매장수: 11
번호: 6  브랜드명: 아멜라떼  매장수: 3
번호: 7  브랜드명: 블랙로럴스커피(Black Laurels Coffee)  매장수: 
번호: 8  브랜드명: 케이빙수(k-빙수)  매장수: 
번호: 9  브랜드명: 광부커피  매장수: 
번호: 10  브랜드명: 루고  매장수: 12
번호: 11  브랜드명: 페르케노  매장수: 7
번호: 12  브랜드명: 아이씨카페  매장수: 
번호: 13  브랜드명: 슈케익하우스(Chou Cake House)  매장수: 24
번호: 14  브랜드명: 레프트뱅크  매장수: 3
번호: 15  브랜드명: 무인상점24  매장수: 
번호: 16  브랜드명: 커피슈퍼  매장수: 
번호: 17  브랜드명: 대박커피  매장수: 
번호: 18  브랜드명: 요거베리  매장수: 
번호: 19  브랜드명: 베이커리로컬  매장수: 2
번호: 20  브랜드명: 훈스커피 앤 디저트  매장수: 
번호: 21  브랜드명: 크로플 각  매장수: 
번호: 22  브랜드명: 1L 맥스커피  매장수: 
번호: 23  브랜드명: 소진담  매장수: 5
번호: 24  브랜드명: 라페브  매장수: 
if문 확인
------------page 변경--------------
page:  41
----------------------------------
if문 확인
번호: 1  브랜드명: 마이요거트립  매장수: 
번호: 2  브랜드명: 레이지모닝  매장수: 7
번호: 3  브랜드명: 오페뜨  매장수: 6
번호: 4  브랜드명: 마망카페  매장수: 2
번호: 5  브랜드명: 브롤빈(BROLLBIN)  매장수: 
번호: 6  브랜드명: 리무소커피  매장수: 
번호: 7  브랜드명: 호두톡톡  매장수: 
번호: 8  브랜드명: 하겐다즈  매장수: 12
번호: 9  브랜드명: 맛있는베이커리  매장수: 
번호: 10  브랜드명: 맛있는커피  매장수: 
번호: 11  브랜드명: 호배기  매장수

번호: 10  브랜드명: 파시야  매장수: 
번호: 11  브랜드명: 크로와플  매장수: 1
번호: 12  브랜드명: CAFE CANBUS(카페캔버스)  매장수: 29
번호: 13  브랜드명: allcha(올차)  매장수: 
번호: 14  브랜드명: 바리스트로  매장수: 8
번호: 15  브랜드명: 브레드카페지  매장수: 
번호: 16  브랜드명: 킥애스(kick ass)  매장수: 
번호: 17  브랜드명: 태미원  매장수: 
번호: 18  브랜드명: 흑운당  매장수: 
번호: 19  브랜드명: 에딕티(Addictea)  매장수: 
번호: 20  브랜드명: 카페락온  매장수: 
번호: 21  브랜드명: 미남커피(MINAM COFFEE)  매장수: 20
번호: 22  브랜드명: 카페임당  매장수: 
번호: 23  브랜드명: 타이티카페(TAETEA CAFE)  매장수: 20
번호: 24  브랜드명: 샌드리카노  매장수: 
if문 확인
------------page 변경--------------
page:  51
----------------------------------
if문 확인
번호: 1  브랜드명: 빈타이(BEAN TIE)  매장수: 9
번호: 2  브랜드명: 타이거슈가(TIGER SUGAR)  매장수: 29
번호: 3  브랜드명: 피스피스  매장수: 3
번호: 4  브랜드명: 1L 커피특별시  매장수: 
번호: 5  브랜드명: 카페야(Cafe Yah)  매장수: 4
번호: 6  브랜드명: 빅스버(VIXBER)  매장수: 
번호: 7  브랜드명: REPLICA COFFEE(레플리카커피)  매장수: 
번호: 8  브랜드명: 이글이글베이글  매장수: 
번호: 9  브랜드명: Breaview(브레뷰)  매장수: 
번호: 10  브랜드명: 시선  매장수: 
번호: 11  브랜드명: 언빌리버블(unbeliebubble)  매장수: 
번호: 12  브랜드명: 비알레티  매장수: 
번호: 13  브랜드명: 호두파야  매장수: 4
번호: 14  브랜드명: 클로버 커피

In [43]:
# 데이터 합치기
df_all = pd.concat([df3,df_a,df_b])
df_all.reset_index(inplace=True)
df_all.to_csv('cafe_info.csv', index=False, encoding='euc-kr')

UnicodeEncodeError: 'euc_kr' codec can't encode character '\xe9' in position 12: illegal multibyte sequence